## CIFAR-10, 100 학습시키기

## Objective

1.[CIFAR -10 Data](https://www.cs.toronto.edu/~kriz/cifar.html) 을 Convolution Neural Network 을 이용해 학습해봅니다.
----
![Imgur](https://i.imgur.com/yy09iLz.png)


- loss 가 가장 작은 model 을 저장합니다.
- 목표 accuracy 는 75% 입니다. 
​

------


In [0]:
import numpy as np 
import tensorflow as tf 
import matplotlib.pyplot as plt
import os

In [2]:
# from tensorflow.python.training import checkpoint_utils as cp
# var_names = cp.list_variables('./ttee/model-0')
# print(var_names)
# print(len(var_names))
# block_4_kernel = cp.load_variable('./ttee/model-0' , 'VGG_block-4/conv/kernel')
# block_4_kernel_1 = cp.load_variable('./ttee/model-0' , 'VGG_block-4/conv_1/kernel')
# print(type(block_4_kernel_1))

sess2 = tf.Session()
saver_var = tf.train.import_meta_graph('./model-16600.meta')
graph_var = tf.get_default_graph()
col_var = graph_var.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
print(len(col_var)) #20개나옴


20


In [3]:
saver_var.restore(sess2, './model-16600')

W0618 07:38:22.172179 139769143691136 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [4]:
#제대로 weight들이 들어있나 확인
for tensor_var in col_var :
    print(tensor_var)

<tf.Variable 'VGG_block-1/conv/kernel:0' shape=(2, 2, 3, 10) dtype=float32_ref>
<tf.Variable 'VGG_block-1/conv/bias:0' shape=(10,) dtype=float32_ref>
<tf.Variable 'VGG_block-2/conv/kernel:0' shape=(2, 2, 10, 50) dtype=float32_ref>
<tf.Variable 'VGG_block-2/conv/bias:0' shape=(50,) dtype=float32_ref>
<tf.Variable 'VGG_block-3/conv/kernel:0' shape=(2, 2, 50, 100) dtype=float32_ref>
<tf.Variable 'VGG_block-3/conv/bias:0' shape=(100,) dtype=float32_ref>
<tf.Variable 'VGG_block-3/conv_1/kernel:0' shape=(2, 2, 100, 100) dtype=float32_ref>
<tf.Variable 'VGG_block-3/conv_1/bias:0' shape=(100,) dtype=float32_ref>
<tf.Variable 'VGG_block-4/conv/kernel:0' shape=(2, 2, 100, 200) dtype=float32_ref>
<tf.Variable 'VGG_block-4/conv/bias:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'VGG_block-4/conv_1/kernel:0' shape=(2, 2, 200, 200) dtype=float32_ref>
<tf.Variable 'VGG_block-4/conv_1/bias:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'FC1/kernel:0' shape=(800, 2000) dtype=float32_ref>
<tf.Variabl

In [0]:
#tf.constant_initializer(var_list[0]) 사용 하기위해 값 추출
var_list = sess2.run(col_var)

# Load Cifar-10 dataset 
 - cifar 10 dataset 을 다운로드 합니다. 
 - normalize 을 수행합니다.

In [6]:
!pip install tensorboardcolab
import tensorboardcolab
#content/tensorboard
#tbc=tensorboardcolab.TensorBoardColab(graph_path='./tensorboard')

Using TensorFlow backend.


In [7]:
# load cifar10 dataset 
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# reshape (None, 1) -> (None)
y_train = np.reshape(y_train, (-1))
y_test = np.reshape(y_test, (-1))

# normalization 
x_train, x_test = x_train/255. , x_test/255.

# N class
n_classes = 10
print('image shape : {}, label shape : {} '.format(x_train.shape, y_train.shape))
print('image shape : {}, label shape : {} '.format(x_test.shape, y_test.shape))
print('train minimun : {}, train_maximum : {} '.format(x_train.min(), x_train.max()))
print('tests minimun : {}, test_maximum : {} '.format(x_test.min(), x_test.max()))

170500096/170498071 [==============================] - 6s 0us/step
image shape : (50000, 32, 32, 3), label shape : (50000,) 
image shape : (10000, 32, 32, 3), label shape : (10000,) 
train minimun : 0.0, train_maximum : 1.0 
tests minimun : 0.0, test_maximum : 1.0 


# DataProvider 

In [0]:
import numpy as np
import random
import copy
np.random.seed(0)
class DataProvider(object):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        self.images_fix = images # fix data저장
        self.labels_fix = labels # fix data저장
        self.len_ = self.images.shape[0] # 총 이미지 갯수저장
        self.len_fix = copy.deepcopy(self.len_)  # fix길이 저장
        self.ind_range = self.images.shape[0] # index
        self.ind = [ x for x in range(self.ind_range)]
        np.random.shuffle(self.ind) # index shuffle
        self.images = self.images[self.ind, :] # shuffle 수행
        self.labels = self.labels[self.ind]
        self.images = list(self.images) #list화 시킴(del() 등 list연산 사용필요)
        self.labels = list(self.labels)

    def next_batch(self, batch_size):
        #fix me#
        if self.len_ <= batch_size :
            ### 해당 epoch의 마지막 batch case ###
            # 1.나머지 모두 내보냄
            out_batch_image = self.images[:][:]
            out_batch_labels = self.labels[:]
            del(self.images[:])
            del(self.labels[:])
            
            # 2.다음 epoch의 shuffle 수행
            self.len_ = self.len_fix
            self.images = self.images_fix
            self.labels = self.labels_fix
            self.ind = [x for x in range(self.ind_range)]
            np.random.shuffle(self.ind)
            self.images = self.images[self.ind,:] # shuffle 수행
            self.labels = self.labels[self.ind]
            self.images = list(self.images)
            self.labels = list(self.labels)
        else : 
            # 일반 batch수행
            out_batch_image, out_batch_labels = self.images[:batch_size][:], self.labels[:batch_size] # slice함
            del(self.images[:batch_size]) # 해당 배치만큼 data삭제
            del(self.labels[:batch_size]) # 해당 배치만큼 data삭제
            self.len_ = self.len_ - batch_size # 길이줄임
            out_batch_labels = np.array(out_batch_labels)
            out_batch_image = np.array(out_batch_image)
        return out_batch_image, out_batch_labels

# Configuration

설계한 모델을 표로 작성합니다. 

- 목표 Receptive Field : ? <br>
- Convolution Phase 후  출력 크기  :  ? <br>


| 층  | 종류|필터 갯수  | 필터 크기 | 스트라이드 | 패딩   | Dropout | output size |
|--- |--- |----|----|----|----|----| ---| 
| ? |?| ?|? |?  | ? |?| ?|


- 모델 설계가 끝나면 간단한 그림을 작성해 아래에 붙여주세요.


예시1) 


- 목표 Receptive Field : 28 <br>
- Convolution Phase 후  출력 크기  :  4 <br>
- Regularization  : L2 
- Batch size : 120
- Learning rate : 0.0001 
- Data normalization : min max normalization 
- Standardization : None 


| 층  | 종류|필터 갯수  | 필터 크기 | 스트라이드 | 패딩   | Dropout | output size |
|--- |--- |----|----|----|----|----| ---| 
| c1 |conv| 64| 3x3| 1  | SAME | None| 32x32 |
| s2 |max-pooling| None| 3x3| 2  | SAME | None|16x16 | 
| c3 |conv| 128| 3x3| 2  | SAME |NOne |16x16 | 
| s4 |max-pooling| None| 3x3| 2  | SAME | None|8 x8 | 
| c5 |conv| 128| 3x3| 2  | SAME | None |8 x8 | 
| s6 |conv| 256| 3x3| 2  | SAME | None |4 x 4 | 
| c7 |conv| 256| 1x1| 2  | SAME | None |4 x 4 | 
| f8 ||| | FC 256  | |  || 
| f8 ||| | Dropout 0.7 | |  || 
| f9 ||| | FC 256  | |  || 
| f9 ||| | Dropout 0.6 | |  || 
| f10||| | FC 10   | |  || 




![Imgur](https://i.imgur.com/yqrIm5u.png)

# Convolution layer
- convolution layer helper function 을 정의합니다.
- 위 설계한 convolution layer 을 구현합니다 

In [0]:
tf.reset_default_graph()
# convolution helper function
def conv(input_xs ,units, k, s, padding, activation, ker_init, bias_init, name):
    layer = tf.layers.Conv2D(filters = units, kernel_size = k, strides = s,
                             padding = padding, activation = activation, use_bias=True,
                             kernel_initializer=ker_init, bias_initializer = bias_init, name = name )(input_xs)
    return layer

In [1]:
# define input placeholder 
xs = tf.placeholder(dtype = tf.float32, shape = [None, 32, 32, 3])
ys = tf.placeholder(dtype = tf.float32, shape = [None])
#ys_one_hot = tf.one_hot()
lr = tf.placeholder(dtype = tf.float32, shape = ())
phase_train = tf.placeholder(tf.bool, shape = (), name = 'phase_train')

NameError: ignored

In [11]:
# Model implementation 
# convolution Neural Network 
# 자신이 설계한 모형을 구현해주세요.
with tf.variable_scope('VGG_block-1') :
    layer = conv(xs, 10, (2,2), (1,1), 'SAME', tf.nn.relu, tf.constant_initializer(var_list[0]), tf.constant_initializer(var_list[1]), 'conv' )
    pooling = tf.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'same')(layer)

with tf.variable_scope('VGG_block-2') :
    layer = conv(pooling, 50, (2,2), (1,1), 'SAME', tf.nn.relu, tf.constant_initializer(var_list[2]), tf.constant_initializer(var_list[3]), 'conv')
    pooling = tf.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'same')(layer)

with tf.variable_scope('VGG_block-3') :
    layer = conv(pooling, 100, (2,2), (1,1), 'SAME', tf.nn.relu, tf.constant_initializer(var_list[4]), tf.constant_initializer(var_list[5]) ,'conv')
    layer = conv(layer, 100, (2,2), (1,1), 'SAME', tf.nn.relu, tf.constant_initializer(var_list[6]), tf.constant_initializer(var_list[7]), 'conv')    
    pooling = tf.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'same')(layer)
    
with tf.variable_scope('VGG_block-4') :
    layer = conv(pooling, 200, (2,2), (1,1), 'SAME', tf.nn.relu, tf.constant_initializer(var_list[8]), tf.constant_initializer(var_list[9]) ,'conv')
    layer = conv(layer, 200, (2,2), (1,1), 'SAME', tf.nn.relu, tf.constant_initializer(var_list[10]), tf.constant_initializer(var_list[11]),'conv')    
    pooling = tf.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'same' )(layer)
    
# with tf.variable_scope('VGG_block-5') :
#     layer = conv(pooling, 400, (3,3), (1,1), 'SAME', tf.nn.relu,'conv')
#     layer = conv(layer,400, (3,3), (1,1), 'SAME', tf.nn.relu, 'conv')    
#     #pooling = tf.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'same' )(layer)
    
top_conv = tf.identity(pooling, 'top_conv') # 마지막 layer 을 top conv 에 넣습니다.
tf.shape(top_conv)

<tf.Tensor 'Shape:0' shape=(4,) dtype=int32>

# Fully Connected Layer
- 설계한 fully connected layer 을 구현합니다.
- dropout 을 적용합니다 

In [0]:
def fc(flat_layer, units, initializer_k, initializer_b, layer_name):
    dense = tf.layers.Dense(units = units, activation = tf.nn.relu,use_bias=True,
                            kernel_initializer = initializer_k, bias_initializer = initializer_b, name = layer_name)(flat_layer)
    return dense

In [13]:
# flat layer 
flatten_layer = tf.layers.flatten(top_conv)
#print("tf.shape(flat_layer) :", tf.shape(flatten_layer))

# fully connected layer 1
#fc_initializer = tf.initializers.he_normal() #<--여기 하는중, 여기부터 하라,, 또한 model의 가중치값을 불러오던지 하라..

fc_layer_1 = fc(flat_layer = flatten_layer, units = 2000, initializer_k = tf.constant_initializer(var_list[12]), initializer_b = tf.constant_initializer(var_list[13]) , layer_name = "FC1" )
fc_layer_1 = tf.layers.dropout(fc_layer_1,rate=0.5, training=phase_train)

# fix me # 자신이 설계한 fully connected layer 을 구현합니다.
fc_layer_2 = fc(flat_layer = fc_layer_1, units = 500, initializer_k = tf.constant_initializer(var_list[14]), initializer_b = tf.constant_initializer(var_list[15]), layer_name = "FC2" )
fc_layer_2 = tf.layers.dropout(fc_layer_2,rate=0.5, training=phase_train)

fc_layer_3 = fc(flat_layer = fc_layer_2, units = 100, initializer_k = tf.constant_initializer(var_list[16]), initializer_b = tf.constant_initializer(var_list[17]), layer_name = "FC3" )

fc_layer_4 = fc(flat_layer = fc_layer_3, units = 10, initializer_k = tf.constant_initializer(var_list[18]), initializer_b = tf.constant_initializer(var_list[19]), layer_name = "FC4" )

logits= tf.identity(fc_layer_4, 'logits')

ys = tf.cast(ys, tf.int32)
loss = tf.losses.sparse_softmax_cross_entropy(ys, logits)


W0618 07:38:39.977081 139769143691136 deprecation.py:323] From <ipython-input-13-3c36e0e311d8>:1: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
W0618 07:38:40.567987 139769143691136 deprecation.py:323] From <ipython-input-13-3c36e0e311d8>:8: dropout (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dropout instead.
W0618 07:38:40.736547 139769143691136 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


#  Loss function 
- loss function 을 정의합니다. L2 regularization 을 사용합니다 

In [0]:
l2_reg = tf.add_n([tf.nn.l2_loss(var) for var in tf.global_variables()])
l2_beta = 5e-4

#loss 
# L2 reularization 
loss = loss + (l2_beta * l2_reg)
loss = tf.identity(loss, name = 'loss')

# Metric

In [0]:
# metric
pred = tf.nn.softmax(logits)
one_hot_label = tf.one_hot(ys, 10)
pred_arg = tf.argmax(pred, axis = 1)
label_arg = tf.argmax(one_hot_label, axis = 1)
eq = tf.cast(tf.equal(pred_arg, label_arg), dtype = tf.float32)
acc = tf.reduce_mean(eq, axis =0)

# Add tensor to Tensorboard 

In [0]:
# add accuracy to tensorboard nodes 
#fix me #
acc_summary = tf.summary.scalar(name='a', tensor=acc)

# add loss to tensorboard nodes 
#fix me #
loss_summary = tf.summary.scalar(name='a', tensor=acc)


#merge all tensorboard nodes 
#fix me #
merged = tf.summary.merge_all()

# Optimizer 

In [0]:
# train_op : adamoptimizer 
#train_op = tf.train.AdamOptimizer(lr).minimize(loss)
#train_op = tf.train.MomentumOptimizer(lr, momentum=0.9).minimize(loss)

# Session open 

In [0]:
sess = tf.Session()

optimizer = tf.train.MomentumOptimizer(lr, momentum=0.9)
train_op = optimizer.minimize(loss)

#초기학습
init_g = tf.global_variables_initializer() # : globalal initializer
init_l = tf.local_variables_initializer() # : local initializer
#sess.run(init_g)
sess.run([init_l,init_g])
#sess.run(tf.variables_initializer(optimizer.variables()))
# saver 
saver = tf.train.Saver()

#Weight Transfer / restore
#saver.restore(sess, './ttee/model-16600')
# optimizer = tf.train.MomentumOptimizer(lr, momentum=0.9)
# train_op = optimizer.minimize(loss)
#sess.run(tf.variables_initializer(optimizer.variables()))
#print(sess.run(optimizer.variables())

# Tensorboard Filewriter 

In [0]:
# tensorboard 
train_writer=tf.summary.FileWriter(logdir='./tensorboard/train')

test_writer=tf.summary.FileWriter(logdir='./tensorboard/test')


# Training 

In [20]:
dataprovider = DataProvider(images=x_train, labels=y_train)
#save_root_folder = #fix me # : models saved folder 

# hparam 
batch_size = 100
min_loss = 1000000.0
learning_rate = 0.00001 # val loss 발산때마다 1/10씩 줄여나감

np.random.seed(0)
#local variable initialize
for i in range(50000):
    batch_xs, batch_ys = dataprovider.next_batch(batch_size)
    # training 
    _= sess.run(train_op, feed_dict = {xs : batch_xs,
                                        ys : batch_ys,
                                        lr : learning_rate,
                                        phase_train : True})
    
    if i % 100 == 0 :
        # Validate validation dataset 
        fetches=[loss, acc, merged]
        val_loss, val_acc, val_merged = sess.run(fetches, feed_dict = {xs : x_test,
                                                                      ys : y_test,
                                                                      phase_train : False})

        # Validate train dataset : extract randomly 10000 samples from train dataset 
        ran = [ x for x in range(0, 50000)]
        nansu = np.random.choice(ran, size = 10000, replace=False)
        train_xtest, train_ytest = x_train[nansu], y_train[nansu]
        train_loss, train_acc, train_merged = sess.run([loss, acc, merged], feed_dict = { xs : train_xtest,
                                                                                          ys : train_ytest,
                                                                                        phase_train : False})
       
        print('step : {} train loss : {:.4f} acc : {:.4f} | Val loss : {:.4f} acc : {:.4f}'.\
        format(i, train_loss, train_acc, val_loss, val_acc))

        # Save Model 
        if val_loss < min_loss : #fix me # : when val_loss < min_loss 
            min_loss = val_loss
            save_path = './ttee/model'
            saver.save(sess, save_path, global_step=i)
            print('model save!')
            
        # Add values to tensorboard 
        train_writer.add_summary(train_merged, i)
        test_writer.add_summary(val_merged, i)
        train_writer.flush()

step : 0 train loss : 0.7688 acc : 0.8141 | Val loss : 1.0729 acc : 0.7032
model save!
step : 100 train loss : 0.7560 acc : 0.8116 | Val loss : 1.0626 acc : 0.7078
model save!
step : 200 train loss : 0.7674 acc : 0.8130 | Val loss : 1.0612 acc : 0.7091
model save!
step : 300 train loss : 0.7458 acc : 0.8220 | Val loss : 1.0609 acc : 0.7091
model save!
step : 400 train loss : 0.7588 acc : 0.8115 | Val loss : 1.0614 acc : 0.7074
step : 500 train loss : 0.7639 acc : 0.8127 | Val loss : 1.0612 acc : 0.7077
step : 600 train loss : 0.7505 acc : 0.8167 | Val loss : 1.0592 acc : 0.7091
model save!
step : 700 train loss : 0.7479 acc : 0.8182 | Val loss : 1.0602 acc : 0.7093
step : 800 train loss : 0.7431 acc : 0.8211 | Val loss : 1.0596 acc : 0.7095
step : 900 train loss : 0.7519 acc : 0.8163 | Val loss : 1.0593 acc : 0.7110
step : 1000 train loss : 0.7505 acc : 0.8151 | Val loss : 1.0623 acc : 0.7086
step : 1100 train loss : 0.7474 acc : 0.8184 | Val loss : 1.0597 acc : 0.7096
step : 1200 trai

W0618 07:39:40.553194 139769143691136 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


step : 1500 train loss : 0.7463 acc : 0.8199 | Val loss : 1.0591 acc : 0.7104
model save!
step : 1600 train loss : 0.7509 acc : 0.8144 | Val loss : 1.0602 acc : 0.7097
step : 1700 train loss : 0.7314 acc : 0.8244 | Val loss : 1.0623 acc : 0.7077
step : 1800 train loss : 0.7399 acc : 0.8205 | Val loss : 1.0598 acc : 0.7097
step : 1900 train loss : 0.7525 acc : 0.8168 | Val loss : 1.0608 acc : 0.7101
step : 2000 train loss : 0.7349 acc : 0.8261 | Val loss : 1.0615 acc : 0.7095
step : 2100 train loss : 0.7487 acc : 0.8184 | Val loss : 1.0631 acc : 0.7075
step : 2200 train loss : 0.7489 acc : 0.8156 | Val loss : 1.0633 acc : 0.7085
step : 2300 train loss : 0.7347 acc : 0.8226 | Val loss : 1.0598 acc : 0.7095
step : 2400 train loss : 0.7395 acc : 0.8215 | Val loss : 1.0610 acc : 0.7095
step : 2500 train loss : 0.7315 acc : 0.8282 | Val loss : 1.0609 acc : 0.7117
step : 2600 train loss : 0.7427 acc : 0.8197 | Val loss : 1.0621 acc : 0.7098
step : 2700 train loss : 0.7512 acc : 0.8157 | Val l

KeyboardInterrupt: ignored